In [1]:
!pip install autokeras

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import autokeras as ak


In [3]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
data = pd.read_csv("/content/gdrive/My Drive/Thesis/dataframe_flow_variable_60min.csv")
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])
data.rename( columns={'Unnamed: 0':'Date_time','225':'DMA_225'}, inplace=True )
data = data.replace(",", ".", regex=True)

In [5]:
# Dataset preparation
data_DMA225 = data[['Date_time','DMA_225','tempC','HeatIndexC','FeelsLikeC','WindChillC','windspeedKmph','sunHour','precipMM','humidity']]
data_DMA225_indexed = data_DMA225.set_index('Date_time')
# Making hourly data 
hourly_data = data_DMA225_indexed.resample('h').sum()
split = int(len(hourly_data) * 0.7)
trainset = hourly_data[:split]
validationset = hourly_data[split:]
testset = hourly_data[["tempC","HeatIndexC","FeelsLikeC","WindChillC","windspeedKmph","sunHour","precipMM","humidity"]]
X_train = trainset[["tempC","HeatIndexC","FeelsLikeC","WindChillC","windspeedKmph","sunHour","precipMM","humidity"]]
valid_X = validationset[['tempC','HeatIndexC','FeelsLikeC','WindChillC','windspeedKmph','sunHour','precipMM','humidity']]
y_train = trainset["DMA_225"]
valid_y = validationset["DMA_225"]
  

In [7]:
# Creating autokeras model:
model = ak.TimeseriesForecaster(
    predict_from=1,
    lookback=12,
    max_trials=1,
    loss='mse', optimizer='adam',
    objective="val_loss",
)

history = model.fit(X_train,y_train,batch_size=100,epochs = 100,validation_data=(valid_X, valid_y))

INFO:tensorflow:Reloading Oracle from existing project ./time_series_forecaster/oracle.json
INFO:tensorflow:Reloading Tuner from ./time_series_forecaster/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
16/16 [==============================] - 8s 132ms/step - loss: 13.5502 - val_loss: 12.5569
Epoch 2/100
16/16 [==============================] - 0s 31ms/step - loss: 10.5010 - val_loss: 8.7896
Epoch 3/100
16/16 [==============================] - 0s 31ms/step - loss: 6.9303 - val_loss: 4.9372
Epoch 4/100
16/16 [==============================] - 1s 34ms/step - loss: 4.2265 - val_loss: 3.1786
Epoch 5/100
16/16 [==============================] - 1s 33ms/step - loss: 3.0452 - val_loss: 2.6502
Epoch 6/100
16/16 [==============================] - 1s 34ms/step - loss: 2.6995 - val_loss: 2.5141
Epoch 7/100
16/16 [==============================] - 0s 31ms/step - loss: 2.6126 - val_loss: 2.4711
Epoch 8/100
16/16 [==============================] - 1s 34ms/step - loss: 2.5695 - val_loss:

INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


INFO:tensorflow:Assets written to: ./time_series_forecaster/best_model/assets


In [8]:
# Predicting on test set:
test_pred = model.predict(testset)
print(test_pred.shape)


22/22 [==============================] - 1s 20ms/step
(651, 1)


 Autokeras is mainly used for mutlivaraite time series analysis. Here the validation loss is greater than train set loss. So it proves auto keras isnt advisble for time series prediction.